In [126]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [127]:
# load data
data = pd.read_csv('data/songs.csv')
data

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,emotion
0,0.214,0.400,0.641,0.000000,0.0656,-6.303,0.0674,81.552,0.3930,0.0
1,0.695,0.445,0.537,0.000017,0.0944,-8.532,0.0400,122.769,0.1310,0.0
2,0.981,0.440,0.040,0.465000,0.1110,-16.887,0.0322,135.965,0.2490,0.0
3,0.751,0.501,0.405,0.000000,0.1050,-5.679,0.0319,109.891,0.4460,0.0
4,0.855,0.411,0.204,0.000000,0.1430,-11.575,0.0578,70.853,0.3370,0.0
...,...,...,...,...,...,...,...,...,...,...
570,0.980,0.630,0.192,0.896000,0.1080,-14.102,0.0426,144.904,0.4900,3.0
571,0.405,0.195,0.168,0.972000,0.0865,-22.176,0.0377,102.540,0.0271,3.0
572,0.753,0.517,0.349,0.806000,0.1200,-17.350,0.0553,128.484,0.1010,3.0
573,0.708,0.199,0.247,0.839000,0.1660,-18.179,0.0436,67.968,0.0337,3.0


In [128]:
# split data into features and labels
x = data.iloc[:, 0:-1].values.astype(np.float32)
y = data.iloc[:, -1].values.astype(np.float32)

In [129]:
# normalize features
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
x = x.astype(np.float32)

In [130]:
# split dataset into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=6014)

In [131]:
# convert to tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [132]:
# create training and testing datasets
class SongDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

train_dataset = SongDataset(x_train, y_train)
test_dataset = SongDataset(x_test, y_test)

In [133]:
# create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [134]:
# build the network
input_dim = x_train.shape[1]
output_dim = len(set(y_train))

class SongEmotionNetwork(nn.Module):
    def __init__(self, hidden_dim):
        super(SongEmotionNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [153]:
# train model
learning_rate = 0.001
num_epochs = 150
hidden_dim = 100
model = SongEmotionNetwork(hidden_dim)
model = model.float()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        # Forward pass
        outputs = model(features)
        loss = loss_fn(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [1/150], Loss: 5.8723
Epoch [2/150], Loss: 5.2731
Epoch [3/150], Loss: 4.4432
Epoch [4/150], Loss: 3.2936
Epoch [5/150], Loss: 2.1161
Epoch [6/150], Loss: 1.4832
Epoch [7/150], Loss: 1.2465
Epoch [8/150], Loss: 1.1340
Epoch [9/150], Loss: 1.0462
Epoch [10/150], Loss: 0.9821
Epoch [11/150], Loss: 0.9149
Epoch [12/150], Loss: 0.8517
Epoch [13/150], Loss: 0.8023
Epoch [14/150], Loss: 0.7757
Epoch [15/150], Loss: 0.7352
Epoch [16/150], Loss: 0.6897
Epoch [17/150], Loss: 0.6814
Epoch [18/150], Loss: 0.6391
Epoch [19/150], Loss: 0.6172
Epoch [20/150], Loss: 0.5994
Epoch [21/150], Loss: 0.5807
Epoch [22/150], Loss: 0.5590
Epoch [23/150], Loss: 0.5448
Epoch [24/150], Loss: 0.5421
Epoch [25/150], Loss: 0.5289
Epoch [26/150], Loss: 0.5023
Epoch [27/150], Loss: 0.4982
Epoch [28/150], Loss: 0.4923
Epoch [29/150], Loss: 0.4910
Epoch [30/150], Loss: 0.4678
Epoch [31/150], Loss: 0.4602
Epoch [32/150], Loss: 0.4583
Epoch [33/150], Loss: 0.4493
Epoch [34/150], Loss: 0.4410
Epoch [35/150], Loss: 0

In [157]:
# eval model
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for features, labels in test_loader:        
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')

Accuracy of the model on the test set: 86.96%
